# Python Version

The ``powershift`` package requires Python 3. If running this notebook using Python 2, switch the running notebook kernel to Python 3.

# Client Initialisation

In [1]:
import powershift.endpoints as endpoints

If the ``host`` is not specified (left as the default value ``None``), then it is assumed that the client is being used from inside of a pod running in an OpenShift cluster. In this case the ``host`` will be automatically set to ``openshift.default.svc.cluster.local`` and the ``token`` will be read from the file in the container at ``/var/run/secrets/kubernetes.io/serviceaccount/token``. Certificate verification will be performed if a ``host`` has been specified and disabled when determined that the client is being used from inside of a pod.

When the client is being used inside of a pod, it is necessary to add the service account for the project to the appropriate role to be able to use the REST API. To add the service account to the ``view`` role, to allow it to read details via the REST API, but not edit anything, use:

```
oc adm policy add-role-to-group view system:serviceaccounts:myproject
```

Change ``myproject`` to be the actual project name the pod is running in. The service account will only be able to view details related to the current project.

If you need to see all details related to projects that a user can work with, you need to supply a ``token`` for the user. Such a token can be obtained by running:

```
oc whoami --token
```

You may want to ensure that it obtain a new token so that it has the maximum lifetime, as user tokens will expire.

In [2]:
host = None
token = None
verify = None

The ``requests`` library will complain on every request where a untrusted certificate is used even when told not to perform certificate validation. This would be the case for when the OpenShift cluster is setup using self signed certificates, as well as when using the client from inside of a pod. Disable the warnings so it isn't so noisy.

In [3]:
import requests.packages.urllib3

if not host and not verify:
    requests.packages.urllib3.disable_warnings()

Create the actual client.

In [4]:
client = endpoints.Client(host=host, token=token, verify=verify)

# Listing All Projects

In [5]:
from ipy_table import *

In [6]:
projects = client.oapi.v1.projects.get()

In [7]:
headers = [('Project', 'Display Name', 'Description')]
rows = [(project.metadata.name,
         project.metadata.annotations['openshift.io/display-name'],
         project.metadata.annotations['openshift.io/description']
        ) for project in projects.items]
table = headers + rows
make_table(table)
apply_theme('basic')

Project,Display Name,Description
myproject,My Project,Initial developer project


# Listing All Pods

In [8]:
headers = [('Project', 'Pod')]
rows = []
for project in projects.items:
    namespace = project.metadata.name
    pods = client.api.v1.namespaces(namespace=namespace).pods.get()
    for pod in pods.items:
        rows.append((namespace, pod.metadata.name))
table = headers + rows
make_table(table)
apply_theme('basic')

Project,Pod
myproject,powershift-7-i6zgo


# Listing All Routes

In [9]:
headers = [('Project', 'Route', 'URL')]
rows = []
for project in projects.items:
    namespace = project.metadata.name
    routes = client.oapi.v1.namespaces(namespace=namespace).routes.get()
    for route in routes.items:
        host = route.spec.host
        path = route.spec.path or '/'
        url = '%s://%s%s' % (route.spec.tls and 'https' or 'http', host, path)
        rows.append((namespace, route.metadata.name, url))
table = headers + rows
make_table(table)
apply_theme('basic')

Project,Route,URL
myproject,powershift,http://powershift-myproject.127.0.0.1.xip.io/
